In [8]:
import sys
import higra as hg
sys.path.append('../evapo_scripts/Auxiliary/')
from helpsters import *

ModuleNotFoundError: No module named 'higra'

In [2]:
# Function from https://github.com/waldnerf/decode/blob/main/postprocessing/instance_segmentation.py

def InstSegm(extent, boundary, t_ext=0.4, t_bound=0.2):
    """
    INPUTS:
    extent : extent prediction
    boundary : boundary prediction
    t_ext : threshold for extent
    t_bound : threshold for boundary
    OUTPUT:
    instances
    """

    # Threshold extent mask
    ext_binary = np.uint8(extent >= t_ext)

    # Artificially create strong boundaries for
    # pixels with non-field labels
    input_hws = np.copy(boundary)
    input_hws[ext_binary == 0] = 1

    # Create the directed graph
    size = input_hws.shape[:2]
    graph = hg.get_8_adjacency_graph(size)
    edge_weights = hg.weight_graph(
        graph,
        input_hws,
        hg.WeightFunction.mean)

    tree, altitudes = hg.watershed_hierarchy_by_dynamics(
        graph,
        edge_weights)

    # Get individual fields
    # by cutting the graph using altitude
    instances = hg.labelisation_horizontal_cut_from_threshold(
        tree,
        altitudes,
        threshold=t_bound).astype(float)

    instances[ext_binary == 0] = np.nan

    return instances

In [5]:
# load masked and unmasked .vrt files and run InstSegm algorythm

paths = ['/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/vrt/chipsvrt.vrt',
         '/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/vrt/masked_chipsvrt.vrt']


ds = gdal.Open(paths[0])
bandNumber = ds.RasterCount
bands = []
for i in range(bandNumber):
    bands.append(ds.GetRasterBand(i+1).ReadAsArray())
cube = np.dstack(bands)



classified = InstSegm(cube[0,:,:], cube[1,:,:],0.4,0.2)

# def exportNPtoVRTextent(array, vrt):
#     ''''''
#     if type(vrt) == osgeo.gdal.Dataset:
#         continue
#     else:
#         vrt = gdal.Open(vrt)
 
#     gtiff_driver = gdal.GetDriverByName('GTiff')
    


#     out_ds = gtiff_driver.Create(storPath + fileName, numberOfXpixels, numberOfYpixels, 1, gdal.GDT_Int32)
#     out_ds.SetGeoTransform(getGeoTransFromNC(ncfile))
#     out_ds.SetProjection(getCRS_WKTfromNC(ncfile))
#     out_ds.GetRasterBand(1).WriteArray(arr)

#     out_ds.GetRasterBand(1).SetDescription(bandname)
#     del out_ds





NameError: name 'hg' is not defined

In [4]:
type(ds)

osgeo.gdal.Dataset